In [ ]:
# Trinidad Marroquin
# qsq707

## Lab 7A

<em>Lab 7 consists of two exercises on relational and document-oriented databases.</em>

In this exercise, you will read in an sqlite database ```sf_food_program_db.sqlite``` that contains data on San Francisco restaurants and health inspections. Be sure this database file is included in the same directory as this notebook. This data comes from [PADJO at Stanford University](http://2015.padjo.org/briefs/data_notes/sql-explore-sf-restaurant-data/). This data has three tables:

* ```businesses``` - data about each restaurant
* ```inspections``` - data about every health inspection made at a restaurant
* ```violations``` - data about every violation that a restaurant has raised

Familiarize yourself with the database by printing out the tables. The common column across the three tables is ```business_id```. You will run a series of SQL statements to answer some questions. 

**Tasks**

The code to connect to the sqlite database has already been written for you. Complete the following tasks.

1. Identify the top 10 restaurants in San Francisco have the most number of health code violations recorded in this database. Run a SQL query using ```read_sql()``` to get the results in a ```DataFrame```. The result should include the business_id, name, address, latitude, longitude, and count of the number of violations. What were the nature of those violations?

2. Create a Folium map with markers for the top 50 restaurants in San Francisco have the most number of health code violations recorded in this database. For the markers, include the name, the address, and the number of health code violations. Are many of these restaurants clustered together in geographically? If so, where in town do you see several restaurants with high numbers of health code violations?

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import folium

# The sample data is provided as an sqlite database file. This code will open the file and you will be able
# start querying the database immediately.
connection = sqlite3.connect( 'sf_food_program_db.sqlite' )

# List all tables in the database
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)
print(tables)

# Preview data from the tables to understand their structure
businesses = pd.read_sql("SELECT * FROM businesses LIMIT 5;", connection)
print("Inspecting businesses table")
print(businesses)

inspections = pd.read_sql("SELECT * FROM inspections LIMIT 5;", connection)
print("Inspecting inspections table")
print(inspections)

violations = pd.read_sql("SELECT * FROM violations LIMIT 5;", connection)
print("Inspecting violations table")
print(violations)


          name
0   businesses
1  inspections
2   violations
Inspecting businesses table
   business_id                                name  \
0           10                    Tiramisu Kitchen   
1           17                GEORGE'S COFFEE SHOP   
2           19               Nrgize Lifestyle Cafe   
3           24  OMNI S.F. Hotel - 2nd Floor Pantry   
4           29                       CHICO'S PIZZA   

                         address           city  postal_code   latitude  \
0                  033 Belden Pl  San Francisco        94104  37.791116   
1               2200 OAKDALE Ave           S.F.        94124  37.741086   
2   1200 Van Ness Ave, 3rd Floor  San Francisco        94109  37.786848   
3  500 California St, 2nd  Floor  San Francisco        94104  37.792888   
4                    131 06th St           S.F.        94103  37.774722   

    longitude  phone_number TaxCode  business_certificate application_date  \
0 -122.403816           NaN     H24              779059.0 

In [16]:
# Enter your query here to identify the top 10 restaurants in San Francisco with the most number of violations
query = """
SELECT b.business_id, b.name, b.address, b.latitude, b.longitude, COUNT(v.ViolationTypeID) AS violation_count
FROM violations v
JOIN businesses b ON v.business_id = b.business_id
GROUP BY b.business_id
ORDER BY violation_count DESC
LIMIT 10;
"""
top_10_violations = pd.read_sql(query, connection)
print("Printing top 10 restaurants with the most number of violations")
print(top_10_violations)

# Let's analyze the nature of the violations:
# Adjusting the SQL query based on the corrected column name 'description'
query_nature_of_violations_corrected = """
WITH Top10Restaurants AS (
    SELECT b.business_id, b.name, COUNT(v.ViolationTypeID) AS violation_count
    FROM violations v
    JOIN businesses b ON v.business_id = b.business_id
    GROUP BY b.business_id
    ORDER BY violation_count DESC
    LIMIT 10
)
SELECT 
    t.business_id, 
    t.name, 
    v.description AS violation_description, 
    v.risk_category, 
    COUNT(v.ViolationTypeID) AS violation_type_count
FROM Top10Restaurants t
JOIN violations v ON t.business_id = v.business_id
GROUP BY t.business_id, v.description, v.risk_category
ORDER BY t.business_id, violation_type_count DESC;
"""

# Running query and loading the result into a DataFrame
violation_nature_df = pd.read_sql(query_nature_of_violations_corrected, connection)

# Analyzing the entire dataset for violation nature, focusing on patterns in risk categories and violation types
violation_summary = violation_nature_df.groupby(
    ['violation_description', 'risk_category']
).agg(
    total_occurrences=('violation_type_count', 'sum'),
    unique_restaurants=('business_id', 'nunique')
).reset_index().sort_values(by='total_occurrences', ascending=False)

# Display the first 20 rows of the summarized analysis
print("-------------------------------------------------------")
print("Printing a summary of the violation for each restaurant")
violation_summary.head(20)


# In comments here, describe the nature of those violations

'''
Key Findings from the Expanded Analysis:

    Most Frequent Violations:
        Unclean or degraded floors, walls, or ceilings:
            Occurred 39 times across 10 unique restaurants.
        Unclean nonfood contact surfaces:
            Occurred 32 times across 9 unique restaurants.
        Unapproved or unmaintained equipment or utensils:
            Occurred 31 times across 10 unique restaurants.

    Moderate and High Risk Violations:
        Inadequately cleaned or sanitized food contact surfaces (Moderate Risk):
            Occurred 28 times across 10 restaurants.
        High risk vermin infestation:
            Occurred 16 times across 6 restaurants.
        High risk food holding temperature:
            Occurred 16 times across 9 restaurants.
        Unclean hands or improper use of gloves (High Risk):
            Occurred 9 times across 5 restaurants.

    Repetitive Moderate Risk Issues:
        Moderate risk vermin infestation:
            Occurred 9 times across 6 restaurants.
        Improper thawing methods and Inadequate food safety knowledge (Moderate Risk):
            Occurred 11 and 9 times respectively across multiple restaurants.

    Low Risk Violations:
        These involve operational inefficiencies like:
            Improper or defective plumbing (9 occurrences, 5 restaurants).
            Improper storage of equipment (8 occurrences, 3 restaurants).
            Unclean hands or improper use of gloves (High Risk, but with operational focus).

Observations:

    Repetitive Issues: Many restaurants suffer from recurring issues, particularly unclean surfaces and equipment maintenance. These suggest poor hygiene protocols or insufficient cleaning routines.
    High Risk Violations: Despite being fewer in number, high-risk violations (like vermin infestations and improper food holding temperatures) are critical to address for public safety.
    Moderate Risk Violations: These are quite frequent and often relate to food handling practices, like food contamination and improper thawing. They indicate training gaps or lack of monitoring.

Conclusion:

This expanded analysis highlights that while low-risk operational issues are the most common, high-risk violations still occur with significant frequency. Addressing these violations requires improving employee training, hygiene protocols, and operational monitoring to ensure compliance with health standards.
'''

Printing top 10 restaurants with the most number of violations
   business_id                     name                     address  \
0         1833  Rogue Ales Public House                673 Union St   
1        64176  Dragon River Restaurant             5045 GEARY Blvd   
2        32698            Hunan Cafe #2            4450 CABRILLO St   
3        71385    Yama Sushi Restaurant            850 Holloway Ave   
4        37882               First Cake             5901 GEARY Blvd   
5         3123              Les Joulins                 44 Ellis St   
6         4621          Louie's Dim Sum            1242 Stockton St   
7        75139         Hakka Restaurant  4401 A Cabrillo St Suite A   
8         3115         Yee's Restaurant              1131 Grant Ave   
9        71065        Chutney USA, Inc.                511 Jones St   

    latitude   longitude  violation_count  
0  37.800254 -122.410447               49  
1  37.780523 -122.473580               44  
2  37.773438 -122.50665

,violation_description,risk_category,total_occurrences,unique_restaurants
32,Unclean or degraded floors walls or ceilings,Low Risk,39,10
31,Unclean nonfood contact surfaces,Low Risk,32,9
29,Unapproved or unmaintained equipment or utensils,Low Risk,31,10
17,Inadequately cleaned or sanitized food contact...,Moderate Risk,28,10
6,Improper food storage,Low Risk,26,10
2,Foods not protected from contamination,Moderate Risk,23,8
20,Moderate risk food holding temperature,Moderate Risk,22,9
11,Inadequate and inaccessible handwashing facili...,Moderate Risk,19,9
33,Unclean or unsanitary food contact surfaces,High Risk,19,9
4,High risk vermin infestation,High Risk,16,6


In [27]:
# SQL query to find the top 50 restaurants with the most violations
query_top_50 = """
SELECT b.business_id, b.name, b.address, b.latitude, b.longitude, COUNT(v.ViolationTypeID) AS violation_count
FROM violations v
JOIN businesses b ON v.business_id = b.business_id
GROUP BY b.business_id
ORDER BY violation_count DESC
LIMIT 50;
"""

# Run the SQL query and load the result into a DataFrame
top_50_violations = pd.read_sql(query_top_50, connection)

# Check for NaN values in latitude or longitude
missing_location = top_50_violations[top_50_violations[['latitude', 'longitude']].isna().any(axis=1)]
print("Restaurants with Missing Location Data:")
print(missing_location)

# Filter out rows with NaN latitude or longitude values
top_50_violations = top_50_violations.dropna(subset=['latitude', 'longitude'])

# Create a Folium map centered around San Francisco
sf_map = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# Add markers for each of the top 50 restaurants (after filtering out NaNs)
for index, row in top_50_violations.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=(
            f"<strong>{row['name']}</strong><br>"
            f"Address: {row['address']}<br>"
            f"Violations: {row['violation_count']}"
        ),
    ).add_to(sf_map)

# Display the map
sf_map

Restaurants with Missing Location Data:
    business_id                          name                     address  \
5          3123                   Les Joulins                 44 Ellis St   
7         75139              Hakka Restaurant  4401 A Cabrillo St Suite A   
22        69883          Yin Da Wonton Noodle             648 Pacific Ave   
23        68390             Mission's Kitchen             2738 Mission St   
33        77693                  SH Dream Inc               253 Church St   
34        71314          Sunmoon Market, Inc.             638 Broadway St   
35        70281   Shai Lai Seafood Restaurant             6253 GEARY Blvd   
49        75230  Taqueria Dos Amigos & Market              1342 Irving St   

    latitude  longitude  violation_count  
5        NaN        NaN               38  
7        NaN        NaN               36  
22       NaN        NaN               32  
23       NaN        NaN               32  
33       NaN        NaN               31  
34      

In [26]:


# In comments here, respond to the question on if there seems to be parts of town where there 
# are number of these restaurants clustered together
'''
There appears to be a cluster of restaurants in the North East corner of San Fransisco with health code violations.
There is also a smaller cluster in the North West corner of San Fransisco with health code violations
'''

Restaurants with Missing Location Data:
    business_id                          name                     address  \
5          3123                   Les Joulins                 44 Ellis St   
7         75139              Hakka Restaurant  4401 A Cabrillo St Suite A   
22        69883          Yin Da Wonton Noodle             648 Pacific Ave   
23        68390             Mission's Kitchen             2738 Mission St   
33        77693                  SH Dream Inc               253 Church St   
34        71314          Sunmoon Market, Inc.             638 Broadway St   
35        70281   Shai Lai Seafood Restaurant             6253 GEARY Blvd   
49        75230  Taqueria Dos Amigos & Market              1342 Irving St   

    latitude  longitude  violation_count  
5        NaN        NaN               38  
7        NaN        NaN               36  
22       NaN        NaN               32  
23       NaN        NaN               32  
33       NaN        NaN               31  
34      

'\nThere appears to be a cluster of restaurants in the North East corner of San Fransisco with health code violations.\nThere is also a smaller cluster in the North West corner of San Fransisco with health code violations\n'